###Requirements

In [ ]:
#Install required libraries
!pip install transformers seqeval unidecode
!pip install -q accelerate peft trl bitsandbytes safetensors tokenizers
!pip install python-rdrsegmenter
!pip install Markdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 512.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=9fbd04e30a566404b62a9540cde67c1a722b48378539f8d260329add6187e8d2
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
#Mount Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Inference Model

In [ ]:
#Link checkpoint drive: https://drive.google.com/drive/folders/18Vs4eng4Qmk6gROC9P8pqaG0mp5dqIdK?usp=sharing
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('VietAI/vit5-base')
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/checkpoint/checkpoint-17500")
# model.to("cuda") ## sử dụng GPU thì dùng dòng này
model.to('cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=768, out_features=768, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): lora.Linear(
                (base_layer): Linear(in_features=768, out_

In [ ]:
def generateAns(sentence):
  encoding = tokenizer(sentence, return_tensors="pt", max_length=1024)
  input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
  outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      max_length=1024,
  )
  labels = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  return labels

###Run

In [ ]:
import re
import pandas as pd

def split_sentences(text):
    sentences = re.split(r'[.!?;]|[\n]', text)
    return sentences

def find_phrase_positions(sentence, phrase):
    # Loại bỏ dấu câu từ câu
    sentence_cleaned = sentence.translate(str.maketrans('', '', string.punctuation))

    # Tách câu thành danh sách các từ
    words = sentence_cleaned.split()

    # Tách cụm từ thành danh sách các từ
    phrase_words = phrase.split()

    # Duyệt qua từng vị trí trong câu
    for i in range(len(words) - len(phrase_words) + 1):
        # Kiểm tra nếu các từ liên tiếp từ vị trí i giống với cụm từ
        if words[i:i + len(phrase_words)] == phrase_words:
            start_pos = i
            end_pos = i + len(phrase_words) - 1
            return start_pos, end_pos

    if sentence == phrase:
      start_pos = 0
      end_pos = len(words) - 1
      return start_pos, end_pos

    return None

def remove_whitespace(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        cleaned_sentence = sentence.strip()
        cleaned_sentences.append(cleaned_sentence)
    return cleaned_sentences

# def getPosSentence(text, sentences):
#     word_positions = [(m.start(), m.end()) for m in re.finditer(r'\S+', text)]

#     word_positions_in_sentences = []
#     start_index = 0
#     for sentence in sentences:
#         sentence_length = len(re.findall(r'\S+', sentence))
#         word_positions_in_sentence = list(range(start_index, start_index + sentence_length))
#         word_positions_in_sentences.append(word_positions_in_sentence)
#         start_index += sentence_length

#     return word_positions_in_sentences

# def count_word_positions(sentences):
#     word_positions_in_sentences = []
#     for sentence in sentences:
#         # Tách câu thành các từ
#         words = sentence.split()
#         # Tạo danh sách chứa vị trí các từ trong câu
#         word_positions = list(range(len(words)))
#         # Thêm danh sách vị trí của từng câu vào danh sách tổng
#         word_positions_in_sentences.append(word_positions)
#     return word_positions_in_sentences

# def getTxt(input_string):
#   itms = input_string.split(',')
#   result_list = []
#   for itm in itms:
#       colon_index = itm.find(':')
#       phrase = itm[colon_index + 1:].strip()
#       result_list.append(phrase)
#   return result_list

# def getPos(index, target_phrases, sentence_position_pairs, i=(1,2)):
#   sentence = sentence_position_pairs[index][0]
#   sentence_indices = sentence_position_pairs[index][i]
#   indices = []
#   for target_phrase in target_phrases:
#       phrase_indices = []
#       phrase_words = target_phrase.split()
#       for i in range(len(sentence) - len(phrase_words) + 1):
#           if sentence[i:i+len(phrase_words)] == phrase_words:
#               phrase_indices.append(sentence_indices[i:i+len(phrase_words)])
#       indices.append(phrase_indices)
#   return indices

# def extract_number_pairs(lst):
#     result = []
#     for item in lst:
#         if isinstance(item, list) and item:  # Kiểm tra xem item có phải là một danh sách không rỗng
#             if len(item) == 1 and isinstance(item[0], list) and all(isinstance(i, int) for i in item[0]):
#                 result.append([item[0][0], item[0][-1]])  # Chỉ append đầu và cuối của item[0]
#             else:
#                 result.extend(extract_number_pairs(item))
#     return result

Input text

In [ ]:
# Đưa đoạn văn vào đây

# text = """Phòng chống lây nhiễm HIV, chủ yếu thông qua các chương trình trao đổi kim tiêm và tình dục an toàn,
# là một chiến dịch quan trọng để kiểm soát sự lây lan của căn bệnh này.
# Tăng cường việc nhận thức và các biện pháp phòng ngừa đối với người dân cũng đóng một vai trò quan trọng.
# Liệu pháp gen được đề nghị là biện pháp khả thi để ngăn ngừa hoặc điều trị nhiễm HIV. Vắc-xin HIV. Điều trị mổ. suy giảm miễn dịch"""

text = """Lé mắt hay lác mắt là hiện tương không cân bằng và thiếu hợp thị giữa hai mắt"""

In [ ]:
import string
sentences = split_sentences(text)
sentences = remove_whitespace(sentences)
sentences
# positions = getPosSentence(text, sentences)
# sencPos = count_word_positions(sentences)
# word_lists = []

# # Lặp qua từng câu trong danh sách câu
# for sentence in sentences:
#     # Loại bỏ các dấu câu từ câu
#     sentence_without_punctuation = sentence.translate(str.maketrans('', '', string.punctuation.replace('-', '')))
#     # Tách từ của câu và đưa vào mảng riêng biệt
#     word_list = sentence_without_punctuation.split()
#     # Thêm mảng từ vào danh sách tổng
#     word_lists.append(word_list)
# sentence_position_pairs = list(zip(word_lists, positions, sencPos))
# pd.DataFrame(sentence_position_pairs)

['Lé mắt hay lác mắt là hiện tương không cân bằng và thiếu hợp thị giữa hai mắt']

In [ ]:
# ans = []
nested_list = []
for index, sent in enumerate(sentences):
    temp = generateAns(sent)
    # ans.append(temp)
    nested_list.append(temp.split(", "))
    print(temp)

# Hậu xử lý
flat_list = []

for sublist in nested_list:
    for item in sublist:
        flat_list.append(item)

# phrase = []
# for i in ans:
#   phrase.append(getTxt(i))
# output = []
# for index in range(len(ans)):
#   if ans[index] == '':
#     output.append('')
#   output.append(getPos(index, phrase[index], sentence_position_pairs,2)) # 1 cho vị trí theo đoạn, 2 cho vị trí theo câu

# # if '' in ans:
# #     ans.remove('')
# number_pairs = extract_number_pairs(output)

# pairs = list(zip(sentences, flat_list, number_pairs))

# pd.DataFrame(pairs)

SYMPTOM_AND_DISEASE: Lé mắt, SYMPTOM_AND_DISEASE: lác mắt, SYMPTOM_AND_DISEASE: thiếu hợp thị giữa hai mắt


In [ ]:
def get_words_after_colon(nested_list):
    result = []
    for sublist in nested_list:
        words = []
        for item in sublist:
            match = re.search(r'(?<=: ).*', item)
            if match:
                words.append(match.group())
        result.append(words)
    return result

# In kết quả
ans = get_words_after_colon(nested_list)
ans

[['Lé mắt', 'lác mắt', 'thiếu hợp thị giữa hai mắt']]

In [ ]:
for i in range(len(ans)):
  s = sentences[i]
  print(s)
  for j in ans[i]:
    if j == '':
      print("None")
    pos = find_phrase_positions(s, j)
    if pos:
      start_pos, end_pos = pos
      print(f"{j}-{(start_pos, end_pos)}")

Lé mắt hay lác mắt là hiện tương không cân bằng và thiếu hợp thị giữa hai mắt
Lé mắt-(0, 1)
lác mắt-(3, 4)
thiếu hợp thị giữa hai mắt-(12, 17)


### Eval


In [ ]:
import json
import pandas as pd


In [ ]:
#Read test set
with open('/content/drive/MyDrive/khoaluan/dataset/test_data/Model/qa_test1.json', 'r', encoding='utf-8') as file:
      testDt = json.load(file)
evalWordData = pd.DataFrame(testDt)
evalWordData = evalWordData.rename(columns={'question':'source_text', 'response':'target_text'})

In [ ]:
evalWordData

,source_text,target_text
0,"Viêm dạ_dày , rối_loạn thần_kinh thực_vật , ga...","SYMPTOM_AND_DISEASE: Viêm dạ_dày, SYMPTOM_AND_..."
1,Trẻ 10 tháng bị nổi hạch sau tai có sao không ?,SYMPTOM_AND_DISEASE: nổi hạch
2,Xuất_hiện cục thịt lồi và sưng vết khâu tầng s...,"SYMPTOM_AND_DISEASE: cục thịt lồi, SYMPTOM_AND..."
3,Sắt tự_do trong máu cao cần điều_trị thế_nào ?,SYMPTOM_AND_DISEASE: Sắt tự_do trong máu cao
4,Nên mổ tinh_hoàn ẩn cho bé lúc_nào ?,medical_procedure: mổ tinh_hoàn ẩn
...,...,...
995,Đi_đại_tiện trong phân có hạt màu trắng đục sa...,SYMPTOM_AND_DISEASE: Đi_đại_tiện trong phân có...
996,Bị giãn phế_quản xơ thuỳ phổi tái đi tái lại s...,SYMPTOM_AND_DISEASE: giãn phế_quản xơ thuỳ phổ...
997,"Tiểu rắt , nhức lỗ tiểu có phải bị viêm niệu_đ...","SYMPTOM_AND_DISEASE: Tiểu rắt, SYMPTOM_AND_DIS..."
998,Nguyên_nhân thiếu nước trong cơ_thể là gì ?,SYMPTOM_AND_DISEASE: thiếu nước trong cơ_thể


In [ ]:
sentence = "Chỉ_số ALT cao 98 U / L liệu có nguy_hiểm không ?"
generateAns(sentence)

''

In [ ]:
# #Create a list of texts
# eval_input_data = evalWordData['source_text']
# eval_input_sequences = []

# for sentence in eval_input_data:
#   eval_input_sequences.append(" ".join(sentence))

# #Create a list of targets
# eval_output_data = evalWordData['target_text']
# eval_output_sequences = []

# for sentence in eval_output_data:
#   eval_output_sequences.append(" ".join(sentence))

In [ ]:
eval_input_sequences = evalWordData['source_text']
eval_output_sequences = evalWordData['target_text']

In [ ]:
def convert_to_bio(source_text, target_text):
    words = source_text.split()
    labels = ['O'] * len(words)  # Khởi tạo tất cả nhãn là 'O'

    for annotation in target_text.split(','):
        annotation = annotation.strip()  # Loại bỏ khoảng trắng thừa
        if ': ' not in annotation:
            continue  # Bỏ qua nếu annotation không chứa dấu phân cách ': '

        label, entities = annotation.split(': ', 1)
        entity_list = [entity.strip() for entity in entities.split(' và ')]  # Tách các phần

        for entity in entity_list:
            entity_words = entity.split()
            for i in range(len(words) - len(entity_words) + 1):
                if words[i:i+len(entity_words)] == entity_words:
                    labels[i] = 'B-' + label
                    for j in range(1, len(entity_words)):
                        labels[i + j] = 'I-' + label
                    break
    return list(labels)
    # return list(zip(words, labels))


In [ ]:
bio_data = []

# for i in range(len(evalWordData)):
for i in range(1000):
    source_text = eval_input_sequences[i]
    target_text = eval_output_sequences[i]
    bio_format = convert_to_bio(source_text, target_text)
    bio_data.append(bio_format)

In [ ]:
len(bio_data)

1000

In [ ]:
real_time = []
for i in range(1000):
    source_text = eval_input_sequences[i]
    output_text = generateAns(source_text)
    bio_format = convert_to_bio(source_text, output_text)
    real_time.append(bio_format)

In [ ]:
len(real_time)

1000

In [ ]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

y_true = bio_data
y_pred = real_time
f1_score(y_true, y_pred)
classification_report(y_true, y_pred)
accuracy_score(y_true, y_pred)

0.9186443433029909

In [ ]:
import markdown
output = markdown.markdown(classification_report(y_true, y_pred))
print(output)


<pre><code>                 precision    recall  f1-score   support
</code></pre>
<p>SYMPTOM_AND_DISEASE       0.80      0.81      0.80      1398
               drug       0.75      0.74      0.74       119
  medical_procedure       0.75      0.72      0.74       217</p>
<pre><code>      micro avg       0.79      0.79      0.79      1734
      macro avg       0.76      0.76      0.76      1734
   weighted avg       0.79      0.79      0.79      1734
</code></pre>
